In [84]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [90]:
data2=pd.read_csv('Technology_Firm_Stock_Price.csv')
data=pd.read_csv('TDY-ROP.csv')
data2['Date'] = pd.to_datetime(data2['Date'])
data['Date'] = data2['Date']
data = data.set_index('Date')
x = data.last('191D')

In [91]:
print(x)

            signal      pred  prev_spread   next_spread
Date                                                   
2020-03-26       0  0.500070     0.500867  [0.44797474]
2020-03-27       1  0.498277     0.447975  [0.45634673]
2020-03-30       1  0.497631     0.456347  [0.47782696]
2020-03-31       0  0.471454     0.477827  [0.41921803]
2020-04-01       1  0.467713     0.419218  [0.40844632]
...            ...       ...          ...           ...
2020-09-28       1  0.177503     0.130753  [0.15878733]
2020-09-29       1  0.176809     0.158787  [0.17907133]
2020-09-30       1  0.185629     0.179071  [0.20373571]
2020-10-01       0  0.195470     0.203736  [0.20424342]
2020-10-02       1  0.210889     0.204243  [0.22286919]

[133 rows x 4 columns]


In [92]:
x['signal'] = x['signal'].replace(0, -1)

/var/folders/8q/hyqzpcrx357g9qd6yqfg924h0000gn/T/ipykernel_29364/1567213730.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['signal'] = x['signal'].replace(0, -1)


In [93]:
data2['Date'] = pd.to_datetime(data2['Date'])

In [94]:
class PairsTradingStrategy:
    def __init__(self, stock1_data, stock2_data, signals, initial_budget=100000):
        self.stock1_data = stock1_data
        self.stock2_data = stock2_data
        self.signals = signals
        self.initial_budget = initial_budget
        self.current_signal_index = 0
        self.budget = initial_budget
        self.inPosition = 0
        self.current_date = stock1_data.index[0]

    def next(self):
        signal = self.signals.iloc[self.current_signal_index]

        print("Date:", self.current_date, "Budget:", self.budget)
        #print(self.budget)

        
        if self.inPosition == 1:
            if signal.iloc[0] == -1:
                # Liquidate positions
                self.budget += self.stock1_data.iloc[self.current_signal_index, 0]
                self.budget += self.stock2_data.iloc[self.current_signal_index, 0]
                self.inPosition = 0
                
        elif self.inPosition == 0:
            if signal.iloc[0] == -1:
                # Enter new positions
                x = min(self.budget // 2, int(self.budget // (2 * self.stock1_data.iloc[self.current_signal_index, 0])))
                self.budget -= x * self.stock1_data.iloc[self.current_signal_index, 0]
                self.budget += x * self.stock2_data.iloc[self.current_signal_index, 0]
                self.inPosition = -1
                
            elif signal.iloc[0] == 1:
                # Enter new positions
                x = min(self.budget // 2, int(self.budget // (2 * self.stock1_data.iloc[self.current_signal_index, 0])))
                self.budget += x * self.stock1_data.iloc[self.current_signal_index, 0]
                self.budget -= x * self.stock2_data.iloc[self.current_signal_index, 0]
                self.inPosition = 1
                

        elif self.inPosition == -1:
            if signal.iloc[0] == 1:
                # Liquidate positions
                self.budget += self.stock1_data.iloc[self.current_signal_index, 0]
                self.budget += self.stock2_data.iloc[self.current_signal_index, 0]
                self.inPosition = 0
                

        self.current_signal_index += 1
        
        self.current_date = self.stock1_data.index[self.current_signal_index]

        if self.current_signal_index == len(self.signals) - 1:
            # Close all positions when all signals have been processed
            if self.inPosition == 1:
                self.budget += self.stock1_data.iloc[self.current_signal_index, 0]
                self.budget += self.stock2_data.iloc[self.current_signal_index, 0]
            elif self.inPosition == -1:
                self.budget += self.stock1_data.iloc[self.current_signal_index, 0]
                self.budget += self.stock2_data.iloc[self.current_signal_index, 0]

            self.inPosition = 0

        return self.budget

def run_backtest(stock1_data, stock2_data, signals, initial_budget):
    strategy = PairsTradingStrategy(stock1_data, stock2_data, signals, initial_budget)
    
    # Iterate over each signal to simulate the backtest
    for _ in range(len(signals)):
        final_budget = strategy.next()

    return final_budget

if __name__ == "__main__":
    # Assuming your data variable is available
    stock1_data = data2[['Date', 'TDY']].set_index("Date")
    stock2_data = data2[['Date', 'ROP']].set_index("Date")
    signals = x[['signal']]

    #ANET-PANW,


    initial_budget = 100

    final_budget = run_backtest(stock1_data, stock2_data, signals, initial_budget)

    print("Final Budget:", final_budget)


Date: 2020-01-02 00:00:00 Budget: 100
Date: 2020-01-03 00:00:00 Budget: 100.0
Date: 2020-01-06 00:00:00 Budget: 814.4609375
Date: 2020-01-07 00:00:00 Budget: 822.018798828125
Date: 2020-01-08 00:00:00 Budget: 1535.9675598144531
Date: 2020-01-09 00:00:00 Budget: 1545.6405334472656
Date: 2020-01-10 00:00:00 Budget: 1545.6405334472656
Date: 2020-01-13 00:00:00 Budget: 1545.6405334472656
Date: 2020-01-14 00:00:00 Budget: 1545.6405334472656
Date: 2020-01-15 00:00:00 Budget: 2280.351287841797
Date: 2020-01-16 00:00:00 Budget: 2309.8414306640625
Date: 2020-01-17 00:00:00 Budget: 2309.8414306640625
Date: 2020-01-21 00:00:00 Budget: 2309.8414306640625
Date: 2020-01-22 00:00:00 Budget: 3062.975616455078
Date: 2020-01-23 00:00:00 Budget: 3030.5050354003906
Date: 2020-01-24 00:00:00 Budget: 3030.5050354003906
Date: 2020-01-27 00:00:00 Budget: 3760.7910766601562
Date: 2020-01-28 00:00:00 Budget: 3792.07373046875
Date: 2020-01-29 00:00:00 Budget: 3792.07373046875
Date: 2020-01-30 00:00:00 Budget: 37

In [83]:
x[x['signal']==-1]

,signal,pred,prev_spread,next_spread
Date,,,,
